![](https://memesbams.com/wp-content/uploads/2017/11/sheldon-sarcasm-meme.jpg)

https://www.kaggle.com/danofer/sarcasm
<div class="markdown-converter__text--rendered"><h3>Context</h3>

<p>This dataset contains 1.3 million Sarcastic comments from the Internet commentary website Reddit. The dataset was generated by scraping comments from Reddit (not by me :)) containing the <code>\s</code> ( sarcasm) tag. This tag is often used by Redditors to indicate that their comment is in jest and not meant to be taken seriously, and is generally a reliable indicator of sarcastic comment content.</p>

<h3>Content</h3>

<p>Data has balanced and imbalanced (i.e true distribution) versions. (True ratio is about 1:100). The
corpus has 1.3 million sarcastic statements, along with what they responded to as well as many non-sarcastic comments from the same source.</p>

<p>Labelled comments are in the <code>train-balanced-sarcasm.csv</code> file.</p>

<h3>Acknowledgements</h3>

<p>The data was gathered by: Mikhail Khodak and Nikunj Saunshi and Kiran Vodrahalli for their article "<a href="https://arxiv.org/abs/1704.05579" rel="nofollow">A Large Self-Annotated Corpus for Sarcasm</a>". The data is hosted <a href="http://nlp.cs.princeton.edu/SARC/0.0/" rel="nofollow">here</a>.</p>

<p>Citation:</p>

<pre><code>@unpublished{SARC,
  authors={Mikhail Khodak and Nikunj Saunshi and Kiran Vodrahalli},
  title={A Large Self-Annotated Corpus for Sarcasm},
  url={https://arxiv.org/abs/1704.05579},
  year=2017
}
</code></pre>

<p><a href="http://nlp.cs.princeton.edu/SARC/0.0/readme.txt" rel="nofollow">Annotation of files in the original dataset: readme.txt</a>.</p>

<h3>Inspiration</h3>

<ul>
<li>Predicting sarcasm and relevant NLP features (e.g. subjective determinant, racism, conditionals, sentiment heavy words, "Internet Slang" and specific phrases). </li>
<li>Sarcasm vs Sentiment</li>
<li>Unusual linguistic features such as caps, italics, or elongated words. e.g., "Yeahhh, I'm sure THAT is the right answer".</li>
<li>Topics that people tend to react to sarcastically</li>
</ul></div>

In [1]:
! pip install -q pyspark==3.1.2 spark-nlp

     |████████████████████████████████| 212.4 MB 65 kB/s 
     |████████████████████████████████| 133 kB 19.9 MB/s 
     |████████████████████████████████| 198 kB 37.4 MB/s 


In [2]:
import sys
import time
import sparknlp

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[8]") \
    .config("spark.driver.memory","6G") \
    .config("spark.driver.maxResultSize", "1G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "800M")\
    .config("spark.jars.packages", 'com.johnsnowlabs.nlp:spark-nlp_2.11:2.6.0') \
    .getOrCreate()

In [3]:
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.3.4
Apache Spark version:  3.1.2


In [18]:
! wget -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sarcasm/train-balanced-sarcasm.csv -P /tmp

--2021-12-02 07:08:19--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sarcasm/train-balanced-sarcasm.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.169.96
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.169.96|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/tmp/train-balanced-sarcasm.csv’ not modified on server. Omitting download.



In [5]:
from pyspark.sql import SQLContext

sql = SQLContext(spark)

trainBalancedSarcasmDF = spark.read.option("header", True).option("inferSchema", True).csv("/tmp/train-balanced-sarcasm.csv")
trainBalancedSarcasmDF.printSchema()

# Let's create a temp view (table) for our SQL queries
trainBalancedSarcasmDF.createOrReplaceTempView('data')

sql.sql('SELECT COUNT(*) FROM data').collect()

root
 |-- label: integer (nullable = true)
 |-- comment: string (nullable = true)
 |-- author: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- score: string (nullable = true)
 |-- ups: string (nullable = true)
 |-- downs: string (nullable = true)
 |-- date: string (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- parent_comment: string (nullable = true)



[Row(count(1)=1010826)]

In [6]:
sql.sql('select * from data limit 20').show()

+-----+--------------------+------------------+------------------+-----+---+-----+-------+-------------------+--------------------+
|label|             comment|            author|         subreddit|score|ups|downs|   date|        created_utc|      parent_comment|
+-----+--------------------+------------------+------------------+-----+---+-----+-------+-------------------+--------------------+
|    0|          NC and NH.|         Trumpbart|          politics|    2| -1|   -1|2016-10|2016-10-16 23:55:23|Yeah, I get that ...|
|    0|You do know west ...|         Shbshb906|               nba|   -4| -1|   -1|2016-11|2016-11-01 00:24:10|The blazers and M...|
|    0|They were underdo...|          Creepeth|               nfl|    3|  3|    0|2016-09|2016-09-22 21:45:37|They're favored t...|
|    0|"This meme isn't ...|         icebrotha|BlackPeopleTwitter|   -8| -1|   -1|2016-10|2016-10-18 21:03:47|deadass don't kil...|
|    0|I could use one o...|         cush2push|MaddenUltimateTeam|    6| -1|

In [7]:
sql.sql('select label,count(*) as cnt from data group by label order by cnt desc').show()

+-----+------+
|label|   cnt|
+-----+------+
|    1|505413|
|    0|505413|
+-----+------+



In [8]:
sql.sql('select count(*) from data where comment is null').collect()

[Row(count(1)=53)]

In [9]:
df = sql.sql('select label,concat(parent_comment,"\n",comment) as comment from data where comment is not null and parent_comment is not null limit 100000')
print(type(df))
df.printSchema()
print("Amount of rows:", df.count())
df = df.limit(2000) #minimize dataset if you are not running on a cluster
df.show()

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- label: integer (nullable = true)
 |-- comment: string (nullable = true)

Amount of rows: 100000
+-----+--------------------+
|label|             comment|
+-----+--------------------+
|    0|Yeah, I get that ...|
|    0|The blazers and M...|
|    0|They're favored t...|
|    0|deadass don't kil...|
|    0|Yep can confirm I...|
|    0|do you find arian...|
|    0|What's your weird...|
|    0|Probably Sephirot...|
|    0|What to upgrade? ...|
|    0|Probably count Ka...|
|    0|I bet if that mon...|
|    0|James Shields Wil...|
|    0|There's no time t...|
|    0|Team Specific Thr...|
|    0|Ill give you a hi...|
|    0|Star Wars, easy. ...|
|    0|You're adorable.
...|
|    0|He actually acts ...|
|    0|Clinton struggles...|
|    0|Is that the Older...|
+-----+--------------------+
only showing top 20 rows



In [16]:
from pyspark.ml import Pipeline

In [20]:
# from sparknlp.annotator import *
# from sparknlp.common import *
# from sparknlp.base import *

# documentAssembler = DocumentAssembler()\
#     .setInputCol("text")\
#     .setOutputCol("document")

# tokenizer = Tokenizer() \
#     .setInputCols(["document"]) \
#     .setOutputCol("token")

# stemmer = Stemmer() \
#     .setInputCols(["token"]) \
#     .setOutputCol("stem")

    
# sentence_detector = SentenceDetector() \
#     .setInputCols(["document"]) \
#     .setOutputCol("sentence") \
#     .setUseAbbreviations(True)

# normalizer = Normalizer() \
#     .setInputCols(["stem"]) \
#     .setOutputCol("normalized")

# finisher = Finisher() \
#     .setInputCols(["normalized"]) \
#     .setOutputCols(["ntokens"]) \
#     .setOutputAsArray(True) \
#     .setCleanAnnotations(True)

# nlp_pipeline = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, stemmer, normalizer, finisher])
# nlp_model = nlp_pipeline.fit(df)
# processed = nlp_model.transform(df).persist()
# processed.count()
# processed.show()

In [ ]:
train, test = processed.randomSplit(weights=[0.7, 0.3], seed=123)
print(train.count())
print(test.count())

1401
599


In [ ]:
from pyspark.ml import feature as spark_ft

stopWords = spark_ft.StopWordsRemover.loadDefaultStopWords('english')
sw_remover = spark_ft.StopWordsRemover(inputCol='ntokens', outputCol='clean_tokens', stopWords=stopWords)
tf = spark_ft.CountVectorizer(vocabSize=500, inputCol='clean_tokens', outputCol='tf')
idf = spark_ft.IDF(minDocFreq=5, inputCol='tf', outputCol='idf')

feature_pipeline = Pipeline(stages=[sw_remover, tf, idf])
feature_model = feature_pipeline.fit(train)

train_featurized = feature_model.transform(train).persist()
train_featurized.count()
train_featurized.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|             comment|             ntokens|        clean_tokens|                  tf|                 idf|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|"""Agreed. I thin...|[agr, i, think, w...|[agr, think, issu...|(500,[0,1,7,9,31,...|(500,[0,1,7,9,31,...|
|    0|"""It's kind of h...|[it, kind, of, ha...|[kind, hard, turn...|(500,[4,6,133,135...|(500,[4,6,133,135...|
|    0|"""Mom
Holy shitb...|[mom, holi, shitb...|[mom, holi, shitb...|(500,[414,484],[1...|(500,[414,484],[5...|
|    0|"""People""
Umm, ...|[peopl, umm, he, ...|[peopl, umm, cant...|(500,[7,49],[1.0,...|(500,[7,49],[2.44...|
|    0|"""Play it cool; ...|[plai, it, cool, ...|[plai, cool, plai...|(500,[21,57,77,18...|(500,[21,57,77,18...|
|    0|"""Said it last y...|[said, it, last, ...|[said, last, year...|(500,[1,4,19,25,2...|(500,

In [ ]:
train_featurized.groupBy("label").count().show()
train_featurized.printSchema()

+-----+-----+
|label|count|
+-----+-----+
|    0| 1285|
|    1|  116|
+-----+-----+

root
 |-- label: integer (nullable = true)
 |-- comment: string (nullable = true)
 |-- ntokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- clean_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)
 |-- idf: vector (nullable = true)



In [ ]:
from pyspark.ml import classification as spark_cls

rf = spark_cls. RandomForestClassifier(labelCol="label", featuresCol="idf", numTrees=100)

model = rf.fit(train_featurized)

In [ ]:
test_featurized = feature_model.transform(test)
preds = model.transform(test_featurized)
preds.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|             comment|             ntokens|        clean_tokens|                  tf|                 idf|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|"""Did Hillary Cl...|[did, hillari, cl...|[hillari, clinton...|(500,[27,108,158,...|(500,[27,108,158,...|[92.0320732186918...|[0.92032073218691...|       0.0|
|    0|"""Gingrich
And C...|[gingrich, and, c...|[gingrich, christ...|(500,[13,107,495]...|(500,[13,107,495]...|[92.6783479665537...|[0.92678347966553...|       0.0|
|    0|"""Hey you wanna ...|[hei, you, wanna,...|[hei, wanna, get,...|(500,[5,6,10,16,7...|(500,[5,6,10,16,7...|[92.2620890672296...|[0.92262089067229...|       0.0|
|   

In [ ]:
pred_df = preds.select('comment', 'label', 'prediction').toPandas()

In [ ]:
pred_df.head()

,comment,label,prediction
0,"""""""Did Hillary Clinton break the law?"""" Chaffe...",0,0.0
1,"""""""Gingrich\nAnd Christie will be in charge of...",0,0.0
2,"""""""Hey you wanna get highhh""""""\nOh man, oh man...",0,0.0
3,"""""""QR Code""""\n""For some reason my brain was se...",0,0.0
4,"""""""The Germans bombed Pearl Harbor"""" Not sure ...",0,0.0


In [ ]:
import pandas as pd
from sklearn import metrics as skmetrics
pd.DataFrame(
    data=skmetrics.confusion_matrix(pred_df['label'], pred_df['prediction']),
    columns=['pred ' + l for l in ['0','1']],
    index=['true ' + l for l in ['0','1']]
)

,pred 0,pred 1
true 0,537,0
true 1,62,0


In [ ]:
print(skmetrics.classification_report(pred_df['label'], pred_df['prediction'], 
                                      target_names=['0','1']))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       537
           1       0.00      0.00      0.00        62

    accuracy                           0.90       599
   macro avg       0.45      0.50      0.47       599
weighted avg       0.80      0.90      0.85       599



/Users/vkocaman/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
spark.stop()